In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Wed Nov 28 16:03:35 2018

@author: michaelhaas
"""

import numpy as np
import matplotlib
matplotlib.use('macosx')
import matplotlib.pyplot as plt
from numpy import random as rng
from tqdm import tqdm
from matplotlib.colors import ListedColormap
import matplotlib.animation as animation




### These functions are copied from the Notebook
### See animation code at bottom

cmap = ListedColormap(['white', 'green'])

# Uniform Lattice creation
def normallattice(N, M, value=0):
    '''
    This function returns an N (rows) x M (columns) lattice with identical values value
    '''
    return np.full((N, M), value)

# Random Lattice creation
def random_lattice(N, M):
    '''
    This function returns an N (rows) x M (columns) lattice with randomized spin values 0 or 1
    '''
    return rng.choice((-1, 1), (N, M))

# plot a lattice
def plot_lattice(lattice, ax=None):
    """
    Create a visualization of a lattice 
    """
    if ax is None:
        ax = plt.gca()
    
    edgecolor = 'black' if max(lattice.shape) < 25 else None
    im = ax.pcolormesh(lattice, cmap=cmap, edgecolor=edgecolor, vmax=1, vmin=-1)
    ax.set_aspect('equal')
    return im



def pointChange(lattice,i,j,BornNums,SurvNums,locs,vals):
    if lattice[i][j] == 1:
        for num in SurvNums:
            if neighborSums(lattice,i,j,locs,vals) == num:
                return 1
        return 0
    else:
        for num in BornNums:
            if neighborSums(lattice,i,j,locs,vals) == num:
                return 1
        return 0
    
def neighborSums(lattice, i , j, locs, vals):
    ip = (i + 1) % len(lattice)
    im = (i - 1) % len(lattice)
    jp = (j + 1) % len(lattice[0])
    jm = (j - 1) % len(lattice[0])
    weight = [1 for i in range(8)]
    for k in range(len(locs)):
        weight[locs[k]] = vals[k]
    
    x = (weight[0]*lattice[im][jm] + weight[1]*lattice[i][jm] + weight[2]*lattice[ip][jm] + weight[3]*lattice[im][j] + weight[4]*lattice[ip][j] + weight[5]*lattice[im][jp] + weight[6]*lattice[i][jp] + weight[7]*lattice[ip][jp])
    return x
    
    
def singleStep(lattice, B, S, locs,vals) :
    output = normallattice(len(lattice),len(lattice[0]))
    for i in range(len(lattice)):
        for j in range(len(lattice[i])):
            output[i][j] = pointChange(lattice,i,j,B,S,locs,vals)
    return output





def update_lattice(lattice, T, J, H):
    i, j = random_select(lattice)

    E = local_energy(lattice, i, j, J, H)
    delta_energy = -2*E
    
    if delta_energy < 0:
        flip_spin(lattice, i, j)
    else:
        r = rng.uniform(0, 1)
        boltzman = np.exp(-delta_energy/T)
        if r < boltzman:
            flip_spin(lattice, i, j)

### setup variables
N, M = 100, 100
B = [3]
S = [2,3]
locs = []
vals = []

store_every = 100
Nsteps = 100

history = []
lattice = random_lattice(N, M)

### run many iterations, storing the lattice in history
for t in tqdm(range(Nsteps)):
    singleStep(lattice, B,S,locs, vals) 

    if t % store_every == 0:
        history.append(lattice.copy())

history = np.asarray(history)

### Create an animation
fig, ax = plt.subplots()
im = plot_lattice(history[0], ax=ax)

def updatefig(frame):
    dataImg = history[frame, :-1,:-1]
    im.set_array(np.ravel(dataImg.T))

    return [im]

anim = animation.FuncAnimation(fig, updatefig, np.arange(0, len(history)), repeat=True, interval=30, blit = True)

plt.show()

100%|██████████| 100/100 [00:08<00:00, 11.12it/s]
